In [79]:
import pandas as pd
from pandas.core.frame import DataFrame

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00514/Bias_correction_ucl.csv')

In [3]:
df['Date'] = pd.to_datetime(df['Date'])

### Pre-processing

In [4]:
df.isna().sum()

station              2
Date                 2
Present_Tmax        70
Present_Tmin        70
LDAPS_RHmin         75
LDAPS_RHmax         75
LDAPS_Tmax_lapse    75
LDAPS_Tmin_lapse    75
LDAPS_WS            75
LDAPS_LH            75
LDAPS_CC1           75
LDAPS_CC2           75
LDAPS_CC3           75
LDAPS_CC4           75
LDAPS_PPT1          75
LDAPS_PPT2          75
LDAPS_PPT3          75
LDAPS_PPT4          75
lat                  0
lon                  0
DEM                  0
Slope                0
Solar radiation      0
Next_Tmax           27
Next_Tmin           27
dtype: int64

In [5]:
def handle_missing_data(df: DataFrame) -> DataFrame:
    df_processed = df.copy()
    df_processed = df_processed.dropna(subset=['Next_Tmax', 'Date'])
    df_processed.reset_index(drop=True, inplace=True)
    # missing_cols = df.columns[df.isna().any()].tolist()

    df_date = df_processed['Date']
    df_processed = df_processed.drop(['Date'], axis=1)
    # df_missing = df[['enrollee_id'] + missing_cols]
    # df_non_missing = df.drop(missing_cols, axis=1)

    knn_imputer = KNNImputer(n_neighbors=1)

    X = knn_imputer.fit_transform(df_processed)

    df_processed = pd.DataFrame(X, columns = df_processed.columns)

    df_processed['Date'] = df_date

    return df_processed

In [6]:
df_processed = handle_missing_data(df)

### Feature Engineering

- Extract some informations from Date column

In [7]:
# Month
df_processed['month'] = df_processed.Date.dt.month

# Day of month
df_processed['day_of_month'] = df_processed.Date.dt.day

# Day of week
df_processed['day_of_week'] = df_processed.Date.dt.day_name()

# Year: for data split purpose only
df_processed['year'] = df_processed.Date.dt.year

### Feature Selection

- Remove features with low correlation with Next_Tmax: |corr| < 0.15
- Remove Date

In [8]:
# split categoric and numeric data
numeric_data = list(set(df_processed.columns) - set(['station', 'month', 'day_of_month', 'day_of_week', 'year']))
categoric_data = ['station', 'month', 'day_of_month', 'day_of_week']

In [9]:
# identify columns with low correlation
corrMatrix = df_processed[numeric_data].corr()
columns_to_drop = list(corrMatrix[(np.absolute(corrMatrix['Next_Tmax']) < 0.15)]['Next_Tmax'].index)
columns_to_drop = columns_to_drop + ['Date', 'Next_Tmin', 'Next_Tmax']
columns_to_drop

['lat',
 'LDAPS_PPT1',
 'Solar radiation',
 'lon',
 'Slope',
 'Date',
 'Next_Tmin',
 'Next_Tmax']

In [44]:
X = df_processed.drop(columns_to_drop, axis = 1)
y = df_processed['Next_Tmax']

In [45]:
numeric_data = list(set(numeric_data) - set(columns_to_drop))

### Data partitioning

In [24]:
class AnualTimeSeriesSplit():
    def __init__(self, first_year, n_years):
        self.n_splits = n_years
        self.first_year = first_year
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        indices = np.arange(n_samples)
        for i in range(self.n_splits):
            start = 0
            mid = X[X['year'] == self.first_year + i].index[-1] + 1
            stop = X[X['year'] == self.first_year + i + 1].index[-1] + 1
            yield indices[start: mid], indices[mid: stop]

In [71]:
# SPLIT_ID = int(len(y)*0.7)
# set SPLIT_ID by the end of the first 4 years on the dataset
SPLIT_ID = X[X['year'] != 2017].index[-1] + 1
SPLIT_ID

6185

In [72]:
X_train = X.iloc[:SPLIT_ID]
y_train = y.iloc[:SPLIT_ID]

X_test = X.iloc[SPLIT_ID:]
y_test = y.iloc[SPLIT_ID:]

In [48]:
tscv = TimeSeriesSplit(n_splits=4)
atscv = AnualTimeSeriesSplit(first_year=2013, n_years=3)

## Model creation

In [49]:
def build_model(model=LGBMRegressor()):
    transformers=[('cat_scale', OneHotEncoder(), categoric_data),
                ('num_scale', MinMaxScaler(), numeric_data)]

    preprocessor = ColumnTransformer(transformers=transformers)

    steps = [('preprocessor', preprocessor),
            ('model', model)]

    model_pipeline = Pipeline(steps=steps, verbose=False)
    return model_pipeline

lgb_model = build_model()

In [50]:
scores = cross_val_score(lgb_model, X_train, y_train, cv=atscv, scoring='r2')

In [51]:
print("R2 score: {0:.3f} (+/- {1:.3f})".format(scores.mean(), scores.std()))

R2 score: 0.698 (+/- 0.104)


### LightGBM

In [55]:
gridParams = {
    'model__learning_rate': [0.01, 0.1],
    'model__n_estimators': [24,50,75,100],
    'model__num_leaves': [8,12,16,31,48,72], # large num_leaves helps improve accuracy but might lead to over-fitting
    'model__random_state' : [42],
    'model__subsample' : [0.5, 0.7, 1],
    'model__reg_alpha' : [1,1.2],
    'model__reg_lambda' : [1,1.2,1.4],
}

In [58]:
model = build_model()
gs = GridSearchCV(model, param_grid=gridParams, cv=atscv, scoring='r2', return_train_score=True, verbose=True)

In [59]:
gs.fit(X_train, y_train)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


GridSearchCV(cv=<__main__.AnualTimeSeriesSplit object at 0x000001F0E5CCC070>,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat_scale',
                                                                         OneHotEncoder(),
                                                                         ['station',
                                                                          'month',
                                                                          'day_of_month',
                                                                          'day_of_week']),
                                                                        ('num_scale',
                                                                         MinMaxScaler(),
                                                                         ['LDAPS_RHmax',
                                                                          'LD

In [61]:
gs.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat_scale', OneHotEncoder(),
                                                  ['station', 'month',
                                                   'day_of_month',
                                                   'day_of_week']),
                                                 ('num_scale', MinMaxScaler(),
                                                  ['LDAPS_RHmax', 'LDAPS_WS',
                                                   'LDAPS_PPT2', 'LDAPS_PPT4',
                                                   'DEM', 'LDAPS_LH',
                                                   'LDAPS_PPT3', 'LDAPS_CC1',
                                                   'LDAPS_CC2', 'LDAPS_CC4',
                                                   'LDAPS_RHmin',
                                                   'Present_Tmax',
                                                   'Present_Tmin',
                    

In [81]:

model = build_model(LGBMRegressor(random_state=42, reg_alpha=1.2, reg_lambda=1,
                               subsample=0.5))
# train model
model.fit(X_train, y_train)
# test score
y_predicted = model.predict(X_test)
r2 = r2_score(y_test, y_predicted, multioutput='uniform_average')
rmse = mean_squared_error(y_test, y_predicted, squared=False)
mae = mean_absolute_error(y_test, y_predicted)
print("Test R2 Score: {0:.3f}".format(r2))
print("Test RMSE Score: {0:.3f}".format(rmse))
print("Test MAE Score: {0:.3f}".format(mae))

Test R2 Score: 0.663
Test RMSE Score: 1.828
Test MAE Score: 1.333


### Neural Networks

In [ ]:
MLPReg(random_state=42)